# Script for computing spacings

Reference: [Nikki Loppi thesis](https://spiral.imperial.ac.uk/handle/10044/1/73888)


$$Re = \frac{\rho U}{\nu}$$ 

In [58]:
Re_x = 1e6
U_inf = 1
rho = 1

# Submarine lengths
# In Fidelity Pointwise, find these with Ctrl+Shift+MMB
L_x_all  = 14.29166
L_x_sail = 1.208333
L_x_stern = 0.5

# Solver 
order = 4


In [59]:
nu_all = rho*U_inf*L_x_all/Re_x
Re_x_sail = rho*U_inf*L_x_sail/nu_all
Re_x_stern = rho*U_inf*L_x_stern/nu_all

print("Config file kinematic viscosity: nu = ", nu_all)
print("Sail Reynolds number: Re = ", Re_x_sail)
print("Stern Reynolds number: Re = ", Re_x_stern)

Config file kinematic viscosity: nu =  1.429166e-05
Sail Reynolds number: Re =  84548.12107200983
Stern Reynolds number: Re =  34985.43906026312


## Loppi's $y^+$

### From Loppi's thesis ...
... The boundary layer blocks were generated such that the first P = 4 Gauss-Legendre solution point is at approximately

$$z^+ = \frac{u^\tau\ z}{\mu} = 2.5$$
 with $u\tau = \sqrt{\tau_w}$ being the wall-shear stress reported in reference: Posa and E. Balaras, “A numerical investigation of the wake of an axisym- metric body with appendages,” Journal of Fluid Mechanics, vol. 792, pp. 470– 498, 2016 ...
 

## Low-order $y^+$ spacing

Reference: https://www.cadence.com/en_US/home/tools/system-analysis/computational-fluid-dynamics/y-plus.html

Assumption: Flat-plate boundary layer theory from Frank M. White's Fluid Mechanics 5th edition, page 467.

In [45]:
print("Loppi used y⁺:", 2.5)
y_plus = 2.5

# Overall
C_f = 0.026/Re_x**(1/7)
tau_wall = C_f*0.5*rho*U_inf**2
U_fric = (tau_wall/rho)**0.5
delta_y_all = y_plus*nu_all/U_fric
print("Complete Submarine: ", delta_y_all)
# Sail
C_f = 0.026/Re_x_sail**(1/7)
tau_wall = C_f*0.5*rho*U_inf**2
U_fric = (tau_wall/rho)**0.5
delta_y_sail = y_plus*nu_all/U_fric

print("Sail: ", delta_y_sail)

C_f = 0.026/Re_x_stern**(1/7)
tau_wall = C_f*0.5*rho*U_inf**2
U_fric = (tau_wall/rho)**0.5
delta_y_stern = y_plus*nu_all/U_fric
print("Stern: ", delta_y_stern)


Loppi used y⁺: 2.5
Complete Submarine:  0.0006725311797740908
Sail:  0.0005637375872991005
Stern:  0.0005293030248176952


## High-order $y^+$ spacing

Assuming we use hexahedral elements [Gauss-Legendre quadrature points](https://en.wikipedia.org/wiki/Gauss%E2%80%93Legendre_quadrature), the high-order element may be chosen to be larger than the low-order spacing we just got. Rule of thumb is to multiply the obtained low-order $y^+$ by the element size (anyways we use Flat-plate assumption and these numbers are best obtained after a few simulation runs iteratively). 

In [46]:
# Keeping first point location by Gauss-Legendre quadrature in one place
x_1 = [0, 0.57735, 0.774597, 0.861136, 0.90618]

Normalised_cell_height_multiplier = [2/(1-x) for x in x_1]
print(Normalised_cell_height_multiplier)

[2.0, 4.7320477936827166, 8.872996366507987, 14.402580942504898, 21.317416329140904]


In [47]:
s_all = delta_y_all*Normalised_cell_height_multiplier[order]
s_sail = delta_y_sail*Normalised_cell_height_multiplier[order]
s_stern = delta_y_stern*Normalised_cell_height_multiplier[order]
ar = 5
print("y⁺ spacing: ", s_all, s_sail, s_stern)
print("Aspect ratio: ", ar)
print("x⁺ spacing: ", s_all*ar, s_sail*ar, s_stern*ar)

y⁺ spacing:  0.0143366271535726 0.012017428848840342 0.011283372944312408
Aspect ratio:  5
x⁺ spacing:  0.071683135767863 0.06008714424420171 0.056416864721562036


## Free-shear region

In the free shear region, the cell size increases as $$\eta = l^{3}* Re^{-3/4},$$ where $l$ is the total length of submarine.

In [61]:
eta_all = y_plus*L_x_all* Re_x**(-3/4)*Normalised_cell_height_multiplier[order]
eta_sail = y_plus*L_x_sail* Re_x_sail**(-3/4)*Normalised_cell_height_multiplier[order]
eta_stern = y_plus*L_x_stern* Re_x_stern**(-3/4)*Normalised_cell_height_multiplier[order]

print("eta: ", eta_all, eta_sail, eta_stern)


eta:  0.01926847032390621 0.010390183762311158 0.008333339409416851
